<a href="https://colab.research.google.com/github/L120Git/NLP_Google_Restaurants/blob/main/2_NLP_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##0.Librerías necesarias

In [1]:
#!pip install rarfile
#!pip install unzip
!pip install stop-words
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=18bc46624fe6d94d4e3a589d82ea70ecf5d3715e96305e506e32eddbb3016bef
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Successfully built stop-words
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=34f7c9e2f320f68c29f78fa97903998768bde7d424d7cb4707a1859dfd04a55e
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [2]:
#Librerias necesarias
from google.colab import drive
import numpy as np
import pandas as pd
from google.colab import drive
import random
#import rarfile # para descomprimir archivos
import json
import csv 
#para reducir la dimensión
from sklearn.manifold import TSNE
#para representar
import matplotlib.pyplot as plt
#preprocesado
import unicodedata
import re
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
from nltk.corpus import stopwords
from stop_words import get_stop_words
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('omw-1.4')
import string
from num2words import num2words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# Montamos GDrive para almacenar los datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1. Carga de datos

In [4]:
#cargamos los datos necesarios desde el gDrive
path = '/content/drive/MyDrive/NLP/'

train = pd.read_csv(f'{path}train.csv')
test= pd.read_csv(f'{path}test.csv')

In [5]:
train

,review_text,reviews_sentiment,words,sentiment_label
0,The Big Sur Pizza truly deserves its awards.,5,"['The', 'Big', 'Sur', 'Pizza', 'truly', 'deser...",0
1,I never knew you could be emotional about pizza.,5,"['I', 'never', 'knew', 'you', 'could', 'be', '...",0
2,Amazing clam chowder. Never had so much clam i...,3,"['Amazing', 'clam', 'chowder', '.', 'Never', '...",1
3,Came back for their pork brisket steak (good) ...,5,"['Came', 'back', 'for', 'their', 'pork', 'bris...",0
4,They have coffee and coffee cinnamon cake avai...,4,"['They', 'have', 'coffee', 'and', 'coffee', 'c...",1
...,...,...,...,...
4760,They serve really fresh bread and everything i...,5,"['They', 'serve', 'really', 'fresh', 'bread', ...",0
4761,"The overall experience was fantastic, the bris...",5,"['The', 'overall', 'experience', 'was', 'fanta...",0
4762,The tropical sauce has a slight kick to it but...,3,"['The', 'tropical', 'sauce', 'slight', 'kick',...",1
4763,Love the tacos here. Get the chorizo.,5,"['Love', 'the', 'tacos', 'here', '.', 'Get', '...",0


In [6]:
test

,review_text,reviews_sentiment,words,sentiment_label
0,"Updated review March 11, 2018. Came in tonight...",1,"['Updated', 'review', 'March', '11', ',', '201...",1
1,Get the bacon and a Bloody Mary and you are set.,5,"['Get', 'the', 'bacon', 'and', 'Bloody', 'Mary...",0
2,I recommend the steamed shrimp and ceviche.,5,"['I', 'recommend', 'the', 'steamed', 'shrimp',...",0
3,Ordering process is a bit wonky and our crab c...,4,"['Ordering', 'process', 'is', 'bit', 'wonky', ...",1
4,Definitely satisfies the ramen cravings. We al...,4,"['Definitely', 'satisfies', 'the', 'ramen', 'c...",1
...,...,...,...,...
1584,"Great ramen, awful service.",3,"['Great', 'ramen', ',', 'awful', 'service', '.']",1
1585,Loved the bacon cheese burger wrap! Nachos wer...,5,"['Loved', 'the', 'bacon', 'cheese', 'burger', ...",0
1586,Great place to enjoy hot crab legs and crawfis...,5,"['Great', 'place', 'to', 'enjoy', 'hot', 'crab...",0
1587,My friend and I started with oyster fries and ...,4,"['My', 'friend', 'and', 'I', 'started', 'with'...",1


## 2. Preprocesado
#### Se realizan los siguientes pasos:

> 1.   Normalizar
2.   Eliminar símbolos y caracteres especiales
3.   Eliminar signos de puntuación
4.   Lematizar
5.   Eliminar stopwords




In [7]:
#creamos una función que nos haga toda la limpieza

stop_words = set(stopwords.words("english")) #cargamos las stop.words
lemmatizer = WordNetLemmatizer() #cargamos el lematizador

def clean_review(review):
    # Convertir a minúsculas
    review = review.lower()
    
    # Eliminar acentos, etc
    text = unicodedata.normalize('NFKD', review).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Eliminar los signos de puntuación y caracteres especiales de cada palabra
    token = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenizar las palabras
    words = word_tokenize(token)

    # Lematizar las palabras
    words = [lemmatizer.lemmatize(w) for w in words]

    # Eliminar las stopwords
    words = [w for w in words if not w in stop_words]

    # Unir las palabras en una sola cadena
    clean_review = ' '.join(words)

    return clean_review


####2.1 Preprocesamos el conjunto de train

In [8]:
# Aplicar la función a la columna 'review'
train['review_clean'] = train['review_text'].apply(clean_review)
train
#creamos un df con solo la columna review_clean y sentiment_label
train_clean =train[['review_clean', 'sentiment_label']]
train_clean

,review_clean,sentiment_label
0,big sur pizza truly deserves award,0
1,never knew could emotional pizza,0
2,amazing clam chowder never much clam clam chow...,1
3,came back pork brisket steak good tiramisu als...,0
4,coffee coffee cinnamon cake available outside ...,1
...,...,...
4760,serve really fresh bread everything made scratch,0
4761,overall experience wa fantastic brisket transc...,0
4762,tropical sauce ha slight kick wa good chicken ...,1
4763,love taco get chorizo,0


####2.2 Preprocesamos el conjunto de test

In [9]:
# Aplicar la función a la columna 'review' y añadirla al df para tener todo ahí
test['review_clean'] = test['review_text'].apply(clean_review)

#creamos un df con solo la columna review_clean y sentiment_label
test_clean =test[['review_clean', 'sentiment_label']]
test_clean

,review_clean,sentiment_label
0,updated review march 11 2018 came tonight 13 y...,1
1,get bacon bloody mary set,0
2,recommend steamed shrimp ceviche,0
3,ordering process bit wonky crab cake cooked co...,1
4,definitely satisfies ramen craving also got fr...,1
...,...,...
1584,great ramen awful service,1
1585,loved bacon cheese burger wrap nacho great,0
1586,great place enjoy hot crab leg crawfish,0
1587,friend started oyster fry takoyaki couldnt rea...,1


In [10]:
#observamos el cambio en train
print('Review original: {}'.format(train['review_text'].values[0]))
print('Review procesada: {}'.format(train['review_clean'].values[0]))

Review original: The Big Sur Pizza truly deserves its awards.
Review procesada: big sur pizza truly deserves award


In [11]:
#observamos el cambio en test
print('Review original: {}'.format(test['review_text'].values[0]))
print('Review procesada: {}'.format(test['review_clean'].values[0]))

Review original: Updated review March 11, 2018. Came in tonight with our 13 year old son, we had the boneless wings, wiith 4 different kinds of sauce: Asian Zing, medium hot, mild hot, and Thai curry. Wings were dry, they don't cook then in the sauce they just throw the sauce on.
Review procesada: updated review march 11 2018 came tonight 13 year old son boneless wing wiith 4 different kind sauce asian zing medium hot mild hot thai curry wing dry dont cook sauce throw sauce


In [12]:
# guardamos los dfs en nuestro \content
train_clean.to_csv('train_clean.csv', encoding='utf-8', index=False)
test_clean.to_csv('test_clean.csv', encoding='utf-8', index=False)


In [13]:
#compruebo que los he guardado en mi disco de colab
!ls -lah   

total 644K
drwxr-xr-x 1 root root 4.0K Mar  9 08:52 .
drwxr-xr-x 1 root root 4.0K Mar  9 08:27 ..
drwxr-xr-x 4 root root 4.0K Mar  7 18:12 .config
drwx------ 5 root root 4.0K Mar  9 08:52 drive
drwxr-xr-x 1 root root 4.0K Mar  7 18:14 sample_data
-rw-r--r-- 1 root root 159K Mar  9 08:52 test_clean.csv
-rw-r--r-- 1 root root 463K Mar  9 08:52 train_clean.csv


In [14]:
# lo movemos a nuestro en nuestro google drive
!cp /content/train_clean.csv /content/drive/MyDrive/NLP/train_clean.csv
!cp /content/test_clean.csv /content/drive/MyDrive/NLP/test_clean.csv